# Heap Memory Comparison: Tuples vs Objects vs C Structs

This notebook compares memory usage of different data structures when used in heaps.
Each section can be run independently to avoid overwhelming your system's RAM.

## Setup: Import Libraries and Define Classes

In [1]:
import sys
import ctypes
from heapq import heappush, heappop
import gc
import psutil
import os
from numpy import array
def get_memory_usage():
    """Get current memory usage in MB"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

print(f"Initial memory usage: {get_memory_usage():.2f} MB")

Initial memory usage: 69.18 MB


## Class Definitions

In [2]:
class heapItem:
    __slots__ = ("password_byte", "probability")
    
    def __init__(self, password: str, probability: float, max_length: int):
        enc = password.encode('utf8')
        if len(enc) > (max_length * 2):
            raise ValueError("The password is too long")
        self.password_byte = enc.ljust((max_length * 2), b'\x00')
        self.probability = probability
    
    @property
    def password(self) -> str:
        return self.password_byte.rstrip(b'\x00').decode('utf8')
    
    def __lt__(self, other) -> bool:
        return self.probability < other.probability
    
    def __repr__(self) -> str:
        return f"heapItem({self.password!r},{self.probability})"
    
    def sizeof(self) -> int:
        return (object.__sizeof__(self) +
                sys.getsizeof(self.password_byte) +
                sys.getsizeof(self.probability))

print("heapItem class defined")

heapItem class defined


In [3]:
def create_heap_item_class(password_length: int):
    class HeapItem(ctypes.Structure):
        _fields_ = [
            ("password", ctypes.c_char * password_length),
            ("probability", ctypes.c_float)
        ]
        
        def __init__(self, password, probability):
            super().__init__()
            # Pad or truncate to exact length
            pwd_bytes: bytes = password.encode('utf-8')[:password_length]
            self.password = pwd_bytes.ljust(password_length, b'\0')
            self.probability = probability
        
        def __lt__(self, other) -> bool:
            return self.probability < other.probability
        
        def __sizeof__(self) -> int:
            return ctypes.sizeof(self)
        
        @property
        def password_string(self) -> str:
            return self.password.decode('utf-8').rstrip('\0')
    
    return HeapItem

print("create_heap_item_class function defined")

create_heap_item_class function defined


In [ ]:
class HeapItemNumpy:
    

## Configuration

In [14]:
# Adjust this number based on your available RAM
# Start with 100,000 and increase if your system can handle it
NUM_ITEMS = 1_000_000  # Reduced from 100_000_000 to prevent RAM issues
PASSWORD_MAX_LENGTH = 10

print(f"Testing with {NUM_ITEMS:,} items")
print(f"Memory usage before tests: {get_memory_usage():.2f} MB")

Testing with 1,000,000 items
Memory usage before tests: 75.88 MB


## Test 1: Tuple Heap

In [15]:
print("=== TUPLE HEAP TEST ===")
heap_tuples = []
memory_before = get_memory_usage()

# Create and populate tuple heap
for i in range(NUM_ITEMS):
    heappush(heap_tuples, (float(i), f"password_{i}"))

memory_after_creation = get_memory_usage()
print(f"Memory after creation: {memory_after_creation:.2f} MB")
print(f"Memory used for creation: {memory_after_creation - memory_before:.2f} MB")

# Calculate total size
tup_size = sys.getsizeof(heap_tuples)
for item in heap_tuples:
    tup_size += sys.getsizeof(item)
    tup_size += sys.getsizeof(item[0])  # probability
    tup_size += sys.getsizeof(item[1])  # password string

print(f"Calculated tuple heap size: {tup_size:,} bytes ({tup_size/1024/1024:.2f} MB)")

# Clean up
del heap_tuples
gc.collect()
memory_after_cleanup = get_memory_usage()
print(f"Memory after cleanup: {memory_after_cleanup:.2f} MB")

=== TUPLE HEAP TEST ===
Memory after creation: 230.25 MB
Memory used for creation: 154.38 MB
Calculated tuple heap size: 144,337,618 bytes (137.65 MB)
Memory after cleanup: 80.07 MB


## Test 2: Object Heap (with __slots__)

In [6]:
print("=== OBJECT HEAP TEST ===")
heap_objects = []
memory_before = get_memory_usage()

# Create and populate object heap
for i in range(NUM_ITEMS):
    heappush(heap_objects, heapItem(f"password_{i}", float(i), PASSWORD_MAX_LENGTH))

memory_after_creation = get_memory_usage()
print(f"Memory after creation: {memory_after_creation:.2f} MB")
print(f"Memory used for creation: {memory_after_creation - memory_before:.2f} MB")

# Calculate total size
obj_size = sys.getsizeof(heap_objects)
for obj in heap_objects:
    obj_size += obj.sizeof()

print(f"Calculated object heap size: {obj_size:,} bytes ({obj_size/1024/1024:.2f} MB)")

# Clean up
del heap_objects
gc.collect()
memory_after_cleanup = get_memory_usage()
print(f"Memory after cleanup: {memory_after_cleanup:.2f} MB")
print()

=== OBJECT HEAP TEST ===
Memory after creation: 82.99 MB
Memory used for creation: 10.38 MB
Calculated object heap size: 11,700,984 bytes (11.16 MB)
Memory after cleanup: 75.10 MB



## Test 3: C Struct Heap

In [7]:
print("=== C STRUCT HEAP TEST ===")
heap_struct = []
memory_before = get_memory_usage()

# Create the HeapItem class
HeapItem = create_heap_item_class(PASSWORD_MAX_LENGTH)

# Create and populate struct heap
for i in range(NUM_ITEMS):
    heappush(heap_struct, HeapItem(f"password_{i}", float(i)))

memory_after_creation = get_memory_usage()
print(f"Memory after creation: {memory_after_creation:.2f} MB")
print(f"Memory used for creation: {memory_after_creation - memory_before:.2f} MB")

# Calculate total size
struct_size = sys.getsizeof(heap_struct)
for struct_item in heap_struct:
    struct_size += struct_item.__sizeof__()

print(f"Calculated struct heap size: {struct_size:,} bytes ({struct_size/1024/1024:.2f} MB)")

# Clean up
del heap_struct
del HeapItem
gc.collect()
memory_after_cleanup = get_memory_usage()
print(f"Memory after cleanup: {memory_after_cleanup:.2f} MB")
print()

=== C STRUCT HEAP TEST ===
Memory after creation: 81.72 MB
Memory used for creation: 6.62 MB
Calculated struct heap size: 2,400,984 bytes (2.29 MB)
Memory after cleanup: 76.87 MB



## Performance Test: Heap Operations

In [8]:
import time

print("=== PERFORMANCE COMPARISON ===")
PERF_TEST_SIZE = min(10000, NUM_ITEMS // 10)  # Smaller size for performance test

def time_heap_operations(heap_type_name, create_func, num_items):
    print(f"\n{heap_type_name} Performance:")
    
    # Time creation
    start_time = time.time()
    heap, total_size = create_func(num_items)
    creation_time = time.time() - start_time
    
    print(f"  Creation time: {creation_time:.4f} seconds")
    print(f"  Items created: {len(heap):,}")
    print(f"  Total size: {total_size:,} bytes ({total_size/1024:.2f} KB)")
    
    # Time popping all items
    start_time = time.time()
    popped_count = 0
    while heap:
        heappop(heap)
        popped_count += 1
    pop_time = time.time() - start_time
    
    print(f"  Pop time: {pop_time:.4f} seconds")
    print(f"  Items per second (creation): {num_items/creation_time:.0f}")
    print(f"  Items per second (pop): {popped_count/pop_time:.0f}")
    
    return creation_time, pop_time, total_size

print(f"Performance test will use {PERF_TEST_SIZE:,} items")

=== PERFORMANCE COMPARISON ===
Performance test will use 10,000 items


In [9]:
# Test functions
def create_tuple_heap(num_items):
    heap = []
    for i in range(num_items):
        heappush(heap, (float(i), f"password_{i}"))
    
    total_size = sys.getsizeof(heap)
    for item in heap:
        total_size += sys.getsizeof(item) + sys.getsizeof(item[0]) + sys.getsizeof(item[1])
    
    return heap, total_size

def create_object_heap(num_items):
    heap = []
    for i in range(num_items):
        heappush(heap, heapItem(f"password_{i}", float(i), PASSWORD_MAX_LENGTH))
    
    total_size = sys.getsizeof(heap)
    for obj in heap:
        total_size += obj.sizeof()
    
    return heap, total_size

def create_struct_heap(num_items):
    heap = []
    HeapItem = create_heap_item_class(PASSWORD_MAX_LENGTH)
    for i in range(num_items):
        heappush(heap, HeapItem(f"password_{i}", float(i)))
    
    total_size = sys.getsizeof(heap)
    for struct_item in heap:
        total_size += struct_item.__sizeof__()
    
    return heap, total_size

print("Performance test functions defined")

Performance test functions defined


In [10]:
# Run performance tests
tuple_times = time_heap_operations("Tuple Heap", create_tuple_heap, PERF_TEST_SIZE)
gc.collect()


Tuple Heap Performance:
  Creation time: 0.0064 seconds
  Items created: 10,000
  Total size: 1,424,066 bytes (1390.69 KB)
  Pop time: 0.0034 seconds
  Items per second (creation): 1560787
  Items per second (pop): 2905447


0

In [11]:
object_times = time_heap_operations("Object Heap", create_object_heap, PERF_TEST_SIZE)
gc.collect()


Object Heap Performance:
  Creation time: 0.0101 seconds
  Items created: 10,000
  Total size: 1,175,176 bytes (1147.63 KB)
  Pop time: 0.0068 seconds
  Items per second (creation): 988616
  Items per second (pop): 1461685


0

In [12]:
struct_times = time_heap_operations("C Struct Heap", create_struct_heap, PERF_TEST_SIZE)
gc.collect()


C Struct Heap Performance:
  Creation time: 0.0098 seconds
  Items created: 10,000
  Total size: 245,176 bytes (239.43 KB)
  Pop time: 0.0128 seconds
  Items per second (creation): 1025076
  Items per second (pop): 780844


19

## Summary and Comparison

In [13]:
print("=== FINAL COMPARISON SUMMARY ===")
print(f"Test size: {PERF_TEST_SIZE:,} items")
print()

print("Memory Efficiency (bytes per item):")
print(f"  Tuples:    {tuple_times[2]/PERF_TEST_SIZE:.1f}")
print(f"  Objects:   {object_times[2]/PERF_TEST_SIZE:.1f}")
print(f"  C Structs: {struct_times[2]/PERF_TEST_SIZE:.1f}")
print()

print("Creation Speed (items/second):")
print(f"  Tuples:    {PERF_TEST_SIZE/tuple_times[0]:.0f}")
print(f"  Objects:   {PERF_TEST_SIZE/object_times[0]:.0f}")
print(f"  C Structs: {PERF_TEST_SIZE/struct_times[0]:.0f}")
print()

print("Pop Speed (items/second):")
print(f"  Tuples:    {PERF_TEST_SIZE/tuple_times[1]:.0f}")
print(f"  Objects:   {PERF_TEST_SIZE/object_times[1]:.0f}")
print(f"  C Structs: {PERF_TEST_SIZE/struct_times[1]:.0f}")

print(f"\nFinal memory usage: {get_memory_usage():.2f} MB")

=== FINAL COMPARISON SUMMARY ===
Test size: 10,000 items

Memory Efficiency (bytes per item):
  Tuples:    142.4
  Objects:   117.5
  C Structs: 24.5

Creation Speed (items/second):
  Tuples:    1560787
  Objects:   988616
  C Structs: 1025076

Pop Speed (items/second):
  Tuples:    2905447
  Objects:   1461685
  C Structs: 780844

Final memory usage: 75.88 MB


## Notes and Recommendations

- **Memory Efficiency**: C structs typically use the least memory per item
- **Speed**: Tuples are usually fastest for creation and access
- **Flexibility**: Objects with `__slots__` provide a good balance
- **Large Datasets**: For millions of items, consider using numpy arrays or memory-mapped files

To test with larger datasets, increase `NUM_ITEMS` in the configuration section, but monitor your RAM usage!